In [14]:
import pandas as pd
import numpy as np
from y_engineering import pct_log
from Indicators import *

In [8]:
data = pd.read_csv('AAPL.csv')

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5809 entries, 0 to 5808
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       5809 non-null   object 
 1   Open       5809 non-null   float64
 2   High       5809 non-null   float64
 3   Low        5809 non-null   float64
 4   Close      5809 non-null   float64
 5   Adj Close  5809 non-null   float64
 6   Volume     5809 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 317.8+ KB


In [12]:
data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
5804,2023-02-10,149.460007,151.339996,149.220001,151.009995,151.009995,57409100
5805,2023-02-13,150.949997,154.259995,150.919998,153.850006,153.850006,62199000
5806,2023-02-14,152.119995,153.770004,150.860001,153.199997,153.199997,61707600
5807,2023-02-15,153.110001,155.500000,152.880005,155.330002,155.330002,65669300
5808,2023-02-16,153.509995,156.330002,153.350006,153.710007,153.710007,68031000


In [30]:
data["pct"] = pct_log(data, "Adj Close", log=False)

In [31]:
data["pct_log"] = pct_log(data, "Adj Close")

In [42]:
data.loc[:5000,["Adj Close", "Volume", "pct_log", "pct"]].to_csv("appl_clean_train.csv", index=False)

In [43]:
data.loc[5001:,["Adj Close", "Volume", "pct_log", "pct"]].to_csv("appl_clean_test.csv", index=False)